In [13]:
from pytorch_lightning import Trainer
from catinous.CatsinomModelGramCache import CatsinomModelGramCache
import catinous.CatsinomModelGramCache as catsmodel
from catinous import utils as cutils
from catinous import CatsinomDataset

import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import argparse
import pytorch_lightning as pl
import sklearn 
from sklearn.metrics import confusion_matrix, auc, roc_curve
import torch
import pandas as pd
import seaborn as sns
import pickle
from py_jotools import mut, slurm
import numpy as np
import gc
import hashlib
import dill

%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
CatsinomModelGramCache.get_default_hparams()

{'root_dir': '/project/catinous/cat_data/',
 'datasetfile': 'catsinom_combined_dataset.csv',
 'batch_size': 8,
 'training_batch_size': 8,
 'transition_phase_after': 0.7,
 'cachemaximum': 128,
 'use_cache': True,
 'random_cache': True,
 'balance_cache': True,
 'force_misclassified': False,
 'direction': 'lr->hr',
 'continous': True,
 'noncontinous_steps': 3000,
 'noncontinous_train_splits': ['train', 'base_train'],
 'val_check_interval': 100,
 'base_model': None,
 'run_postfix': '1'}

In [17]:
hparams={'continous':False,
         'datasetfile': 'catsinom_lr_dataset.csv',
         'noncontinous_train_splits': ['base_train'],
         'noncontinous_steps': 103}
# pllogger = cutils.pllogger(hparams)

In [19]:
model, logs, df_cache = catsmodel.trained_model(hparams)

INFO:root:No continous learning, following parameters are invalidated: 
transition_phase_after 
cachemaximum 
use_cache 
random_cache 
force_misclassified 
direction


{'balance_cache': True,
 'base_model': None,
 'batch_size': 8,
 'cachemaximum': 128,
 'continous': False,
 'datasetfile': 'catsinom_lr_dataset.csv',
 'direction': 'lr->hr',
 'force_misclassified': False,
 'noncontinous_steps': 103,
 'noncontinous_train_splits': ['base_train'],
 'random_cache': True,
 'root_dir': '/project/catinous/cat_data/',
 'run_postfix': '1',
 'training_batch_size': 8,
 'transition_phase_after': 0.7,
 'use_cache': False,
 'val_check_interval': 100}


In [18]:
sparams = {
    'binary': '/home/cir/jhofmanninger/env/candid/bin/python',
    'cwd': '/home/cir/jhofmanninger/Projects/catinous',
    'gpu': 1,
    'partition': 'full',
    'memory': 15000,
    'jobname': 'catinous_test'}
slurm.srun(catsmodel.trained_model, [hparams], params=sparams, remote=True)

15815110948576212
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition full /scratch/15815110948576212.job


'/scratch/15815110948576212_res.dll'

In [55]:
dill.load(open('/scratch/158150528649969.dll','rb'))['function'](hparams)

INFO:root:No continous learning, following parameters are invalidated: 
transition_phase_after 
cachemaximum 
use_cache 
random_cache 
force_misclassified 
direction


{'balance_cache': True,
 'base_model': None,
 'batch_size': 8,
 'cachemaximum': 128,
 'continous': False,
 'datasetfile': 'catsinom_lr_dataset.csv',
 'direction': 'lr->hr',
 'force_misclassified': False,
 'noncontinous_steps': 101,
 'noncontinous_train_splits': ['base_train'],
 'random_cache': True,
 'root_dir': '/project/catinous/cat_data/',
 'run_postfix': '1',
 'training_batch_size': 8,
 'transition_phase_after': 0.7,
 'use_cache': False,
 'val_check_interval': 100}


(CatsinomModelGramCache(
   (model): ResNet(
     (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (layer1): Sequential(
       (0): Bottleneck(
         (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu): ReLU(inplace=True)
         (downsa

In [ ]:
/scratch/15815045002145245_res.dll

In [ ]:
model = CatsinomModelGramCache(hparams=hparams, device=torch.device('cuda'))
logger = cutils.pllogger(model.hparams)
trainer = Trainer(gpus=1, max_epochs=1, early_stop_callback=False, logger=logger, val_check_interval=3000, show_progress_bar=True, checkpoint_callback=False)
trainer.fit(model)
torch.save(model.state_dict(), '/project/catinous/trained_models/' + expname + '_run_'+str(i)+'.pt')
save_cache_to_csv(model.trainingscache.cachelist, '/project/catinous/trained_cache/' + expname + '_run_'+str(i)+'.csv')

In [ ]:
slurm.srun(trainer.fit, model, params=sparams, remote=True)

In [7]:
slurm.slurm_params_default

{'binary': '/home/jhofmanninger/anaconda3/envs/candid/bin/python',
 'outputpath': '/home/cir/jhofmanninger/slurmoutput/',
 'jobname': '',
 'days': 0,
 'hours': '0',
 'minutes': '10',
 'memory': 2000,
 'ntasks': 1,
 'cpusptask': 4,
 'qos': 'normal',
 'mailuser': 'johannes.hofmanninger@meduniwien.ac.at',
 'cwd': '/home/jhofmanninger/Projects/catinous',
 'dillfile': '',
 'gpu': 0,
 'partition': 'cir',
 'paths': []}

In [4]:
def save_cache_to_csv(cache, savepath):
    df_cache = pd.DataFrame({'filepath':[ci.filepath for ci in cache], 'label': [ci.label.cpu().numpy()[0] for ci in cache], 'res': [ci.res for ci in cache], 'traincounter': [ci.traincounter for ci in cache]})
    df_cache.to_csv(savepath, index=False, index_label=False)

In [ ]:
hparams = get_default_hparams()
hparams['force_misclassified'] = True
hparams['cachemaximum'] = 128
hparams['continous'] = False
expname = 'continous_random_cache_transphase_force_misclassified_bsize123'
model = CatsinomModelGramCache(argparse.Namespace(**hparams), device = torch.device('cuda'))

basemodel = '/project/catinous/trained_models/lrbase_iterations.pt'
model.load_state_dict(torch.load(basemodel))

logger = pllogging.TestTubeLogger( 'catinous_log_iterations', name=expname)
trainer = Trainer(gpus=1, max_epochs=1, early_stop_callback=False, logger=logger, val_check_interval=10, show_progress_bar=False, checkpoint_callback=False)
trainer.fit(model)

INFO:root:gpu available: True, used: True
INFO:root:VISIBLE GPUS: 0
INFO:root:
              Name               Type Params
0            model             ResNet   24 M
1      model.conv1             Conv2d    9 K
2        model.bn1        BatchNorm2d  128  
3       model.relu               ReLU    0  
4    model.maxpool          MaxPool2d    0  
..             ...                ...    ...
150       model.fc         Sequential    1 M
151     model.fc.0             Linear    1 M
152     model.fc.1        BatchNorm1d    1 K
153     model.fc.2             Linear  513  
154           loss  BCEWithLogitsLoss    0  

[155 rows x 3 columns]
/home/jhofmanninger/anaconda3/envs/candid/lib/python3.6/site-packages/pytorch_lightning/callbacks/pt_callbacks.py:314: RuntimeWarning: Can save best model only with val_loss available, skipping.
  ' skipping.', RuntimeWarning)


In [ ]:
#testing different transition phases

transitionphase = [1.0]

for tp in transitionphase:
    for i in range(2):
        hparams = get_default_hparams()
        hparams['transition_phase_after'] = tp
        
        expname = 'continous_random_cache_transphase_' + str(tp)
        
        model = CatsinomModelGramCache(argparse.Namespace(**hparams), device = torch.device('cuda'))

        model.load_state_dict(torch.load(basemodel))

        logger = pllogging.TestTubeLogger('catinous_log_iterations', name=expname)
        trainer = Trainer(gpus=1, max_epochs=1, early_stop_callback=False, logger=logger, val_check_interval=10, show_progress_bar=False)
        trainer.fit(model)
        torch.save(model.state_dict(), '/project/catinous/trained_models/' + expname + '_run_'+str(i)+'.pt')

        save_cache_to_csv(model.trainingscache.cachelist, '/project/catinous/trained_cache/' + expname + '_run_'+str(i)+'.csv')
        
        trainer = None
        model = None
        trainer = None
        torch.cuda.empty_cache()
        gc.collect()

INFO:root:gpu available: True, used: True
INFO:root:VISIBLE GPUS: 0
/home/mperkonigg/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mperkonigg/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mperkonigg/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mperk

In [ ]:
#testing different cache sizes
cache_size = [16, 128, 256]

for cs in cache_size:
    for i in range(3):
        hparams = get_default_hparams()
        hparams['cachemaximum'] = cs
        
        expname = 'continous_random_cache_cachesize_' + str(cs)
        
        model = CatsinomModelGramCache(argparse.Namespace(**hparams), device = torch.device('cuda'))

        model.load_state_dict(torch.load(basemodel))

        logger = pllogging.TestTubeLogger('catinous_log_iterations', name=expname)
        trainer = Trainer(gpus=1, max_epochs=1, early_stop_callback=False, logger=logger, val_check_interval=10, show_progress_bar=False)
        trainer.fit(model)
        torch.save(model.state_dict(), '/project/catinous/trained_models/' + expname + '_run_'+str(i)+'.pt')

        save_cache_to_csv(model.trainingscache.cachelist, '/project/catinous/trained_cache/' + expname + '_run_'+str(i)+'.csv')

        trainer = None
        model = None
        torch.cuda.empty_cache()